In [7]:
import requests
# URL da API IBGE
URL = "https://servicodados.ibge.gov.br/api/v3/agregados/1288/periodos/1950|1960|1970|1980|1991|2000|2010/variaveis/606?localidades=N3[all]&classificacao=1[all]"

# Função para extrair os dados da API
def extrair_dados():
    response = requests.get(URL)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erro ao acessar API: {response.status_code}")
        return []

In [15]:
import pandas as pd
import json

# Suponha que você tenha um JSON como este:
dados = extrair_dados()

# Explode a camada 'resultados'
resultados = pd.json_normalize(
    dados,
    record_path=['resultados'],
    meta=['id', 'variavel', 'unidade']
)

# Explode a coluna 'classificacoes'
resultados = resultados.explode('classificacoes').reset_index(drop=True)

# Extrai informações da coluna 'classificacoes'
classificacoes_df = pd.json_normalize(resultados['classificacoes'])
resultados = pd.concat([resultados.drop(columns=['classificacoes']), classificacoes_df], axis=1)

# Lida com a lista 'series' e extrai informações tabulares
linha_list = []
for _, row in resultados.iterrows():
    for item in row['series']:
        localidade = item['localidade']
        serie_anos = item['serie']
        
        # Trata categoria de forma segura
        categoria_dict = row.get('categoria', {})
        categoria_id = next(iter(categoria_dict), None)
        categoria_nome = categoria_dict.get(categoria_id) if categoria_id else None

        base_info = {
            'id': row['id'],
            'variavel': row['variavel'],
            'unidade': row['unidade'],
            'categoria_id': categoria_id,
            'categoria_nome': categoria_nome,
            'localidade_id': localidade['id'],
            'localidade_nome': localidade['nome'],
            'nivel_id': localidade['nivel']['id'],
            'nivel_nome': localidade['nivel']['nome']
        }

        for ano, valor in serie_anos.items():
            nova_linha = base_info.copy()
            nova_linha['ano'] = ano
            nova_linha['valor'] = valor
            linha_list.append(nova_linha)

# Cria o DataFrame final
df_final = pd.DataFrame(linha_list)

# Exemplo: ver as primeiras linhas
print(df_final.head())

                                           id   variavel  unidade  \
0  id    606
id      1
Name: 0, dtype: object  População  Pessoas   
1  id    606
id      1
Name: 0, dtype: object  População  Pessoas   
2  id    606
id      1
Name: 0, dtype: object  População  Pessoas   
3  id    606
id      1
Name: 0, dtype: object  População  Pessoas   
4  id    606
id      1
Name: 0, dtype: object  População  Pessoas   

  categoria_id categoria_nome localidade_id localidade_nome nivel_id  \
0         None           None            11        Rondônia       N3   
1         None           None            11        Rondônia       N3   
2         None           None            11        Rondônia       N3   
3         None           None            11        Rondônia       N3   
4         None           None            11        Rondônia       N3   

             nivel_nome   ano    valor  
0  Unidade da Federação  1950    36935  
1  Unidade da Federação  1960    70783  
2  Unidade da Federação  1970

In [16]:
# Salva o DataFrame final em um arquivo Excel
df_final.to_excel('populacao_tabulada.xlsx', index=False)